# CA 4

### The purpose of this project is to learn machine learning methods with Scikit-Learn library.

# Imports

In [58]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Body

# Phase 0 : Data Analysis

In [59]:
df = pd.read_csv("dataset.csv")

In [60]:
df

,type,title,cast,country,release_year,listed_in,description
0,Movie,Dick Johnson Is Dead,NaN,United States,2020,Documentaries,"As her father nears the end of his life, filmm..."
1,TV Show,Blood & Water,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,2021,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,TV Show,Ganglands,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,2021,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,TV Show,Jailbirds New Orleans,NaN,NaN,2021,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,TV Show,Kota Factory,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,2021,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
...,...,...,...,...,...,...,...
11054,TV Show,X-Men: Evolution,"Noel Fisher, Vincent Gale, Christopher Judge, ...",United States,2000,"Action-Adventure, Animation, Kids",X-Men: Evolution features the team as teenager...
11055,TV Show,Smart Guy,"Tahj Mowry, John Jones, Jason Weaver, Essence ...",United States,1996,"Comedy, Coming of Age, Kids",A genius tries to fit in as a high school soph...
11056,TV Show,Disney Kirby Buckets,"Jacob Bertrand, Mekai Curtis, Cade Sutton, Oli...",United States,2014,"Action-Adventure, Comedy, Coming of Age",Welcome to Kirby's world! It's rude and sketchy.
11057,TV Show,Disney Mech-X4,"Nathaniel Potvin, Raymond Cham, Kamran Lucas, ...",Canada,2016,"Action-Adventure, Comedy, Science Fiction",Ryan discovers his ability to control a giant ...


### As seen above , dataset was loaded successfully.

## 1. Dataset information:

In [61]:
for title in df:
    print("In", title, "column :")
    print(df[title].describe())
    print("*"*50)

In type column :
count     11059
unique        2
top       Movie
freq       6131
Name: type, dtype: object
**************************************************
In title column :
count              11059
unique             10957
top       Sister, Sister
freq                   2
Name: title, dtype: object
**************************************************
In cast column :
count                   9694
unique                  9307
top       David Attenborough
freq                      20
Name: cast, dtype: object
**************************************************
In country column :
count              8364
unique              760
top       United States
freq               3105
Name: country, dtype: object
**************************************************
In release_year column :
count    11059.000000
mean      2014.209603
std          8.959517
min       1925.000000
25%       2013.000000
50%       2017.000000
75%       2019.000000
max       2021.000000
Name: release_year, dtype: float64
****

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11059 entries, 0 to 11058
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   type          11059 non-null  object
 1   title         11059 non-null  object
 2   cast          9694 non-null   object
 3   country       8364 non-null   object
 4   release_year  11059 non-null  int64 
 5   listed_in     11059 non-null  object
 6   description   11059 non-null  object
dtypes: int64(1), object(6)
memory usage: 604.9+ KB


## 2. percentage of data lost:

In [63]:
for title in df:
    print("Percentage of data lost in", title, "column :")
    data_lost = df[title].isnull()
    percentage = (data_lost.sum() / data_lost.count()) * 100
    print(percentage)
    print("*"*50)

Percentage of data lost in type column :
0.0
**************************************************
Percentage of data lost in title column :
0.0
**************************************************
Percentage of data lost in cast column :
12.342888145401936
**************************************************
Percentage of data lost in country column :
24.36929197938331
**************************************************
Percentage of data lost in release_year column :
0.0
**************************************************
Percentage of data lost in listed_in column :
0.0
**************************************************
Percentage of data lost in description column :
0.0
**************************************************


# Phase 1 : Preprocess

## 1. Handle Null Data:

### first method: drop rows with at least one Nan value

### second method: fill Nan values with their column's mode.

### chose second method because we want to use other information in rows with at leat one Nan value and we don't want to minify our dataset

In [64]:
for title in df:
    df[title] = df[title].fillna(df[title].mode()[0])

In [65]:
for title in df:
    print("Percentage of data lost in", title, "column :")
    data_lost = df[title].isnull()
    percentage = (data_lost.sum() / data_lost.count()) * 100
    print(percentage)
    print("*"*50)

Percentage of data lost in type column :
0.0
**************************************************
Percentage of data lost in title column :
0.0
**************************************************
Percentage of data lost in cast column :
0.0
**************************************************
Percentage of data lost in country column :
0.0
**************************************************
Percentage of data lost in release_year column :
0.0
**************************************************
Percentage of data lost in listed_in column :
0.0
**************************************************
Percentage of data lost in description column :
0.0
**************************************************


### As seen above , lost data was filled with mode of columns successfully.

## 2. Normalization VS Standardization:

Normalization is a scaling technique in which values are shifted and rescaled so that they end up ranging between 0 and 1. It is also known as Min-Max scaling.

Standardization is another scaling technique where the values are centered around the mean with a unit standard deviation. This means that the mean of the attribute becomes zero and the resultant distribution has a unit standard deviation.


Normalization is good to use when you know that the distribution of your data does not follow a Gaussian distribution. This can be useful in algorithms that do not assume any distribution of the data like K-Nearest Neighbors and Neural Networks.
Standardization, on the other hand, can be helpful in cases where the data follows a Gaussian distribution. However, this does not have to be necessarily true. Also, unlike normalization, standardization does not have a bounding range. So, even if you have outliers in your data, they will not be affected by standardization.


Standardization comes into picture when features of input data set have large differences between their ranges, or simply when they are measured in different measurement units (e.g., Pounds, Meters, Miles … etc).

These differences in the ranges of initial features causes trouble to many machine learning models. For example, for the models that are based on distance computation, if one of the features has a broad range of values, the distance will be governed by this particular feature.

resources: https://www.analyticsvidhya.com/blog/2020/04/feature-scaling-machine-learning-normalization-standardization/
https://builtin.com/data-science/when-and-why-standardize-your-data

### In this project, Normalization and Standardization don't effect on result because we use tree based algorithms(Decision Tree and Random Forrest) for this project. so we choose Normalization to just choose!

In [66]:
df["release_year"]

0        2020
1        2021
2        2021
3        2021
4        2021
         ... 
11054    2000
11055    1996
11056    2014
11057    2016
11058    2008
Name: release_year, Length: 11059, dtype: int64

In [67]:
df["release_year"] = (df["release_year"] - df["release_year"].mean()) / df["release_year"].std()

In [68]:
df["release_year"]

0        0.646285
1        0.757898
2        0.757898
3        0.757898
4        0.757898
           ...   
11054   -1.585979
11055   -2.032431
11056   -0.023394
11057    0.199832
11058   -0.693073
Name: release_year, Length: 11059, dtype: float64

### Normalization by subtract mean and divide by the standard deviation

## 4. Handle listed_in values

### For handle this column, we can separate values or concatenate values.

### We separate listed_in values in to new columns and then remove listed_in column. Because the genres are deferent and most be considered separatly.

In [69]:
df

,type,title,cast,country,release_year,listed_in,description
0,Movie,Dick Johnson Is Dead,David Attenborough,United States,0.646285,Documentaries,"As her father nears the end of his life, filmm..."
1,TV Show,Blood & Water,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,0.757898,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,TV Show,Ganglands,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",United States,0.757898,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,TV Show,Jailbirds New Orleans,David Attenborough,United States,0.757898,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,TV Show,Kota Factory,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,0.757898,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
...,...,...,...,...,...,...,...
11054,TV Show,X-Men: Evolution,"Noel Fisher, Vincent Gale, Christopher Judge, ...",United States,-1.585979,"Action-Adventure, Animation, Kids",X-Men: Evolution features the team as teenager...
11055,TV Show,Smart Guy,"Tahj Mowry, John Jones, Jason Weaver, Essence ...",United States,-2.032431,"Comedy, Coming of Age, Kids",A genius tries to fit in as a high school soph...
11056,TV Show,Disney Kirby Buckets,"Jacob Bertrand, Mekai Curtis, Cade Sutton, Oli...",United States,-0.023394,"Action-Adventure, Comedy, Coming of Age",Welcome to Kirby's world! It's rude and sketchy.
11057,TV Show,Disney Mech-X4,"Nathaniel Potvin, Raymond Cham, Kamran Lucas, ...",Canada,0.199832,"Action-Adventure, Comedy, Science Fiction",Ryan discovers his ability to control a giant ...


In [72]:
max_num_of_values = 0
for item in df["listed_in"]:
    if len(item.split(", ")) > max_num_of_values:
        max_num_of_values = len(item.split(", "))

In [73]:
print("Number of columns we need to separate the listed_in columns :", max_num_of_values)

Number of columns we need to separate the listed_in columns : 5


In [74]:
for i in range(0, max_num_of_values):
    ls = []
    for item in df["listed_in"]:
        if len(item.split(", ")) <= i:
            index = i % len(item.split(", "))
            ls.append(item.split(", ")[index])
        else:
            ls.append(item.split(", ")[i])
    column_name = "genre" + str(i)
    df[column_name] = ls 

In [75]:
df

,type,title,cast,country,release_year,listed_in,description,genre0,genre1,genre2,genre3,genre4
0,Movie,Dick Johnson Is Dead,David Attenborough,United States,0.646285,Documentaries,"As her father nears the end of his life, filmm...",Documentaries,Documentaries,Documentaries,Documentaries,Documentaries
1,TV Show,Blood & Water,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,0.757898,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",International TV Shows,TV Dramas,TV Mysteries,International TV Shows,TV Dramas
2,TV Show,Ganglands,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",United States,0.757898,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,Crime TV Shows,International TV Shows,TV Action & Adventure,Crime TV Shows,International TV Shows
3,TV Show,Jailbirds New Orleans,David Attenborough,United States,0.757898,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",Docuseries,Reality TV,Docuseries,Reality TV,Docuseries
4,TV Show,Kota Factory,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,0.757898,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,International TV Shows,Romantic TV Shows,TV Comedies,International TV Shows,Romantic TV Shows
...,...,...,...,...,...,...,...,...,...,...,...,...
11054,TV Show,X-Men: Evolution,"Noel Fisher, Vincent Gale, Christopher Judge, ...",United States,-1.585979,"Action-Adventure, Animation, Kids",X-Men: Evolution features the team as teenager...,Action-Adventure,Animation,Kids,Action-Adventure,Animation
11055,TV Show,Smart Guy,"Tahj Mowry, John Jones, Jason Weaver, Essence ...",United States,-2.032431,"Comedy, Coming of Age, Kids",A genius tries to fit in as a high school soph...,Comedy,Coming of Age,Kids,Comedy,Coming of Age
11056,TV Show,Disney Kirby Buckets,"Jacob Bertrand, Mekai Curtis, Cade Sutton, Oli...",United States,-0.023394,"Action-Adventure, Comedy, Coming of Age",Welcome to Kirby's world! It's rude and sketchy.,Action-Adventure,Comedy,Coming of Age,Action-Adventure,Comedy
11057,TV Show,Disney Mech-X4,"Nathaniel Potvin, Raymond Cham, Kamran Lucas, ...",Canada,0.199832,"Action-Adventure, Comedy, Science Fiction",Ryan discovers his ability to control a giant ...,Action-Adventure,Comedy,Science Fiction,Action-Adventure,Comedy


### delete listed_in column:

In [77]:
df.drop(labels = "listed_in", axis = 1, inplace = True)

In [78]:
df

,type,title,cast,country,release_year,description,genre0,genre1,genre2,genre3,genre4
0,Movie,Dick Johnson Is Dead,David Attenborough,United States,0.646285,"As her father nears the end of his life, filmm...",Documentaries,Documentaries,Documentaries,Documentaries,Documentaries
1,TV Show,Blood & Water,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,0.757898,"After crossing paths at a party, a Cape Town t...",International TV Shows,TV Dramas,TV Mysteries,International TV Shows,TV Dramas
2,TV Show,Ganglands,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",United States,0.757898,To protect his family from a powerful drug lor...,Crime TV Shows,International TV Shows,TV Action & Adventure,Crime TV Shows,International TV Shows
3,TV Show,Jailbirds New Orleans,David Attenborough,United States,0.757898,"Feuds, flirtations and toilet talk go down amo...",Docuseries,Reality TV,Docuseries,Reality TV,Docuseries
4,TV Show,Kota Factory,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,0.757898,In a city of coaching centers known to train I...,International TV Shows,Romantic TV Shows,TV Comedies,International TV Shows,Romantic TV Shows
...,...,...,...,...,...,...,...,...,...,...,...
11054,TV Show,X-Men: Evolution,"Noel Fisher, Vincent Gale, Christopher Judge, ...",United States,-1.585979,X-Men: Evolution features the team as teenager...,Action-Adventure,Animation,Kids,Action-Adventure,Animation
11055,TV Show,Smart Guy,"Tahj Mowry, John Jones, Jason Weaver, Essence ...",United States,-2.032431,A genius tries to fit in as a high school soph...,Comedy,Coming of Age,Kids,Comedy,Coming of Age
11056,TV Show,Disney Kirby Buckets,"Jacob Bertrand, Mekai Curtis, Cade Sutton, Oli...",United States,-0.023394,Welcome to Kirby's world! It's rude and sketchy.,Action-Adventure,Comedy,Coming of Age,Action-Adventure,Comedy
11057,TV Show,Disney Mech-X4,"Nathaniel Potvin, Raymond Cham, Kamran Lucas, ...",Canada,0.199832,Ryan discovers his ability to control a giant ...,Action-Adventure,Comedy,Science Fiction,Action-Adventure,Comedy


## 3. Handle Categorical Data:

### One-hot Encoding VS Label Encoding:

Label Encoding is very simple and it involves converting each value in a column to a number.

Though label encoding is straight but it has the disadvantage that the numeric values can be misinterpreted by algorithms as having some sort of hierarchy/order in them. This ordering issue is addressed in another common alternative approach called ‘One-Hot Encoding’. In this strategy, each category value is converted into a new column and assigned a 1 or 0 (notation for true/false) value to the column.

resorce: https://towardsdatascience.com/categorical-encoding-using-label-encoding-and-one-hot-encoder-911ef77fb5bd

### We use tree based algorithms and so use both of these Encodings create same result; so we use Label Encoding because it is more simple than One-hot Encoding.

In [79]:
label_encoder = LabelEncoder()

In [80]:
df

,type,title,cast,country,release_year,description,genre0,genre1,genre2,genre3,genre4
0,Movie,Dick Johnson Is Dead,David Attenborough,United States,0.646285,"As her father nears the end of his life, filmm...",Documentaries,Documentaries,Documentaries,Documentaries,Documentaries
1,TV Show,Blood & Water,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,0.757898,"After crossing paths at a party, a Cape Town t...",International TV Shows,TV Dramas,TV Mysteries,International TV Shows,TV Dramas
2,TV Show,Ganglands,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",United States,0.757898,To protect his family from a powerful drug lor...,Crime TV Shows,International TV Shows,TV Action & Adventure,Crime TV Shows,International TV Shows
3,TV Show,Jailbirds New Orleans,David Attenborough,United States,0.757898,"Feuds, flirtations and toilet talk go down amo...",Docuseries,Reality TV,Docuseries,Reality TV,Docuseries
4,TV Show,Kota Factory,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,0.757898,In a city of coaching centers known to train I...,International TV Shows,Romantic TV Shows,TV Comedies,International TV Shows,Romantic TV Shows
...,...,...,...,...,...,...,...,...,...,...,...
11054,TV Show,X-Men: Evolution,"Noel Fisher, Vincent Gale, Christopher Judge, ...",United States,-1.585979,X-Men: Evolution features the team as teenager...,Action-Adventure,Animation,Kids,Action-Adventure,Animation
11055,TV Show,Smart Guy,"Tahj Mowry, John Jones, Jason Weaver, Essence ...",United States,-2.032431,A genius tries to fit in as a high school soph...,Comedy,Coming of Age,Kids,Comedy,Coming of Age
11056,TV Show,Disney Kirby Buckets,"Jacob Bertrand, Mekai Curtis, Cade Sutton, Oli...",United States,-0.023394,Welcome to Kirby's world! It's rude and sketchy.,Action-Adventure,Comedy,Coming of Age,Action-Adventure,Comedy
11057,TV Show,Disney Mech-X4,"Nathaniel Potvin, Raymond Cham, Kamran Lucas, ...",Canada,0.199832,Ryan discovers his ability to control a giant ...,Action-Adventure,Comedy,Science Fiction,Action-Adventure,Comedy


In [81]:
for title in df:
    if(df[title].dtype == "object"):
        df[title] = label_encoder.fit_transform(df[title])

In [82]:
df

,type,title,cast,country,release_year,description,genre0,genre1,genre2,genre3,genre4
0,0,2454,2073,608,0.646285,2886,22,19,20,21,21
1,1,1345,486,430,0.757898,1969,37,69,79,38,69
2,1,3366,7616,608,0.757898,9326,20,37,75,19,39
3,1,4403,2073,608,0.757898,4263,24,51,22,51,23
4,1,4827,5839,254,0.757898,5328,37,54,76,38,55
...,...,...,...,...,...,...,...,...,...,...,...
11054,1,10792,6499,608,-1.585979,10868,2,4,39,2,5
11055,1,7978,8332,608,-2.032431,599,18,16,39,17,18
11056,1,2527,3549,608,-0.023394,9785,2,15,17,2,17
11057,1,2530,6339,59,0.199832,7405,2,15,62,2,17


### Label Encode non-numerical columns by sklearn LableEncoder

# Phase 2 : Predict the Goal and Model Optimization

# Phase 3 : Prediction with Group Learning